In [1]:
import nltk
import pandas as pd
pd.set_option('display.max_colwidth',100)
import re
import string
#nltk.download()
#python -m nltk.downloader all - from command line
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from collections import defaultdict
from nltk import pos_tag
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#Preparing dataset

In [2]:
text=pd.read_csv('https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/research_paper.csv')

In [3]:
text.head()

,Title,Conference
0,Innovation in Database Management: Computer Science vs. Engineering.,VLDB
1,High performance prime field multiplication for GPU.,ISCAS
2,enchanted scissors: a scissor interface for support in cutting and interactive fabrication.,SIGGRAPH
3,Detection of channel degradation attack by Intermediary Node in Linear Networks.,INFOCOM
4,Pinning a Complex Network through the Betweenness Centrality Strategy.,ISCAS


In [4]:
text=text.loc[(text['Conference']=='VLDB') | (text['Conference']=='ISCAS')]

In [5]:
text.head()

,Title,Conference
0,Innovation in Database Management: Computer Science vs. Engineering.,VLDB
1,High performance prime field multiplication for GPU.,ISCAS
4,Pinning a Complex Network through the Betweenness Centrality Strategy.,ISCAS
5,Analysis and Design of Memoryless Interconnect Encoding Scheme.,ISCAS
9,A &#916;&#931; IR-UWB radar with sub-mm ranging capability for human body monitoring systems.,ISCAS


In [ ]:
#Text Preprocessing

In [29]:
def clean_text_into_sentences(text):
    text_output=re.split('\W+',text)
    text_output=[word.lower() for word in text_output]
    text_output=[w for w in text_output if w not in string.punctuation]
    text_output=[w for w in text_output if w.isalpha()]
    stop_words = stopwords.words('english')
    text_output=[w for w in text_output if w not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text_output=" ".join([lemmatizer.lemmatize(w) for w in text_output])
    return text_output

text['cleaned_text']=text['Title'].apply(lambda x: clean_text_into_sentences(x))

In [42]:
# def clean_text_into_tokens(text):
#     text_output=re.split('\W+',text)
#     text_output=[word.lower() for word in text_output]
#     text_output=[w for w in text_output if w not in string.punctuation]
#     text_output=[w for w in text_output if w.isalpha()]
#     stop_words = stopwords.words('english')
#     text_output=[w for w in text_output if w not in stop_words]
#     lemmatizer = WordNetLemmatizer()
#     text_output=[lemmatizer.lemmatize(w) for w in text_output]
#     return text_output

In [6]:
#Count Vectorizer

In [44]:
# cv=CountVectorizer(analyzer=clean_text_into_tokens)
# cv_fit=cv.fit_transform(text['Title'])
# print(cv_fit.shape)
# print(cv.vocabulary_)
# print(cv.get_feature_names())
# df_cv=pd.DataFrame(cv_fit.toarray(),columns=cv.get_feature_names())
# df_cv.head()

# cv_ngram=CountVectorizer(ngram_range=(2,2))
# cv_ngram_fit=cv_ngram.fit_transform(text['cleaned_text'])
# print(cv_ngram_fit.shape)
# print(cv_ngram.vocabulary_)
# print(cv_ngram.get_feature_names())
# df_cv=pd.DataFrame(cv_ngram_fit.toarray(),columns=cv_ngram.get_feature_names())
# df_cv.head()

cv=CountVectorizer()
cv_fit=cv.fit_transform(text['cleaned_text'])
# print(cv_fit.shape)
# print(cv.vocabulary_)
# print(cv.get_feature_names())
df_cv=pd.DataFrame(cv_fit.toarray(),columns=cv.get_feature_names())

df_cv['Conference']=text['Conference'].tolist()

x = df_cv.iloc[:, 0:2682].values
y = df_cv.iloc[:, 2682].values

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.25, random_state=1000)

classifier = LogisticRegression()
classifier.fit(train_x, train_y)
score = classifier.score(test_x, test_y)
print("Accuracy:", score)
Accuracy: 0.796

Accuracy: 0.9409937888198758


In [ ]:
#Tfidf Vectorizer

In [43]:
# tfidf=TfidfVectorizer(analyzer=clean_text_into_tokens)
# tfidf_fit=tfidf.fit_transform(text['Title'])
# print(tfidf_fit.shape)
# print(tfidf.vocabulary_)
# print(tfidf.get_feature_names())
# df_tfidf=pd.DataFrame(tfidf_fit.toarray(),columns=tfidf.get_feature_names())
# df_tfidf.head()

# tfidf_ngram=TfidfVectorizer(ngram_range=(2,2))
# tfidf_ngram_fit=tfidf_ngram.fit_transform(text['cleaned_text'])
# print(tfidf_ngram_fit.shape)
# print(tfidf_ngram.vocabulary_)
# print(tfidf_ngram.get_feature_names())
# df=pd.DataFrame(tfidf_ngram_fit.toarray(),columns=tfidf_ngram.get_feature_names())
# df.head()

tfidf=TfidfVectorizer()
tfidf_fit=tfidf.fit_transform(text['cleaned_text'])
# print(tfidf_fit.shape)
# print(tfidf.vocabulary_)
# print(tfidf.get_feature_names())
df_tfidf=pd.DataFrame(tfidf_fit.toarray(),columns=tfidf.get_feature_names())

df_tfidf['Conference']=text['Conference'].tolist()

x = df_tfidf.iloc[:, 0:2682].values
y = df_tfidf.iloc[:, 2682].values

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.25, random_state=1000)

classifier = LogisticRegression()
classifier.fit(train_x, train_y)
score = classifier.score(test_x, test_y)
print("Accuracy:", score)
Accuracy: 0.796

Accuracy: 0.8975155279503105
